# Import Modules

In [83]:
#pip install collections
#pip install sklearn
#pip install requests
#pip install fuzzywuzzy
#pip install pandas
#pip install numpy


In [84]:
import pandas as pd
import numpy as np
import requests
from collections import Counter
from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.cluster import DBSCAN
import re

#import files
#df = pd.read_csv('cleandata_processed.csv', index_col=0)
df = pd.read_excel(io ='/Users/antoniodrakes/Desktop/Voxcroft/new_data.xlsx')


# Data Cleaning [Lydia & Molly]

Position and Frequency [Lydia]

In [85]:

# to run for whole `dataset use subset = df`
#subset = df
subset = df[:1000]

#create a new empty dataset to add
df_new = pd.DataFrame(columns=df.columns)
#the column with the name or entity name
df_new.rename(columns={'Article_Content_People_AI_Model': 'Name'}, inplace=True)
#the type, either people or entity
df_new.rename(columns={'Article_Content_Entities_AI_Model': 'Identity_Type'}, inplace=True)
#new column with the position of the name relative to the others in the article
df_new['Position'] = np.nan
#new column with the article associated
df_new['Article'] = np.nan
#new column with the frequency of a name in an article
df_new['Frequency'] = np.nan
#new column with Name Cluster
df_new['Cluster'] = np.nan
#new column with Best Name 
df_new['Best Name'] = np.nan
#new column with Probability Score 
df_new['Probability Score'] = np.nan

#separate the strings into arrays
for num in range(0, len(subset)):
    subset.at[num, 'Article_Content_People_AI_Model'] = str(subset['Article_Content_People_AI_Model'][num]).split("|")
    subset.at[num, 'Article_Content_Entities_AI_Model'] = str(subset['Article_Content_Entities_AI_Model'][num]).split("|")
    subset.at[num, 'Article_Subject_Keyword_Identified'] = str(subset['Article_Subject_Keyword_Identified'][num]).split("|")
    subset.at[num, 'Article_Topic_Keyword_Identified'] = str(subset['Article_Topic_Keyword_Identified'][num]).split("|")

    #add the people names
newdf_counter = 0
olddf_counter = -1
article_counter = 0
for entry in subset['Article_Content_People_AI_Model']:
    olddf_counter+=1
    position_counter = 0
    for word in entry:
        if entry.index(word) < position_counter:
            df_new.loc[newdf_counter - position_counter + entry.index(word), 'Position']+=("|"+str(position_counter))
            break
        df_new.loc[newdf_counter, 'Name'] = word
        df_new.loc[newdf_counter, 'Identity_Type'] = 'People'
        df_new.loc[newdf_counter, 'Position'] = str(position_counter)
        df_new.loc[newdf_counter, 'Article'] = article_counter
        df_new.loc[newdf_counter, 'Article_Date_Published'] = subset.loc[olddf_counter, 'Article_Date_Published']
        df_new.loc[newdf_counter, 'Article_Body'] = subset.loc[olddf_counter, 'Article_Body']
        df_new.loc[newdf_counter, 'Article_Source'] = subset.loc[olddf_counter, 'Article_Source']
        df_new.loc[newdf_counter, 'Voice'] = subset.loc[olddf_counter, 'Voice']
        df_new.loc[newdf_counter, 'Article_Themes_AI_Model'] = subset.loc[olddf_counter, 'Article_Themes_AI_Model']
        df_new.at[newdf_counter, 'Article_Subject_Keyword_Identified'] = subset.at[olddf_counter, 'Article_Subject_Keyword_Identified']
        df_new.at[newdf_counter, 'Article_Topic_Keyword_Identified'] = subset.at[olddf_counter, 'Article_Topic_Keyword_Identified']
        newdf_counter+=1
        position_counter+=1
    article_counter+=1

    #add the entity names
newdf_counter = len(df_new)
olddf_counter = -1
article_counter = 0
for entry in subset['Article_Content_Entities_AI_Model']:
    olddf_counter+=1
    position_counter = 0
    for word in entry:
        if entry.index(word) < position_counter:
            df_new.loc[newdf_counter - position_counter + entry.index(word), 'Position']+=("|"+str(position_counter))
            break
        df_new.loc[newdf_counter, 'Name'] = word
        df_new.loc[newdf_counter, 'Identity_Type'] = 'Entity'
        df_new.loc[newdf_counter, 'Position'] = str(position_counter)
        df_new.loc[newdf_counter, 'Article'] = article_counter
        df_new.loc[newdf_counter, 'Article_Date_Published'] = subset.loc[olddf_counter, 'Article_Date_Published']
        df_new.loc[newdf_counter, 'Article_Body'] = subset.loc[olddf_counter, 'Article_Body']
        df_new.loc[newdf_counter, 'Article_Source'] = subset.loc[olddf_counter, 'Article_Source']
        df_new.loc[newdf_counter, 'Voice'] = subset.loc[olddf_counter, 'Voice']
        df_new.loc[newdf_counter, 'Article_Themes_AI_Model'] = subset.loc[olddf_counter, 'Article_Themes_AI_Model']
        df_new.at[newdf_counter, 'Article_Subject_Keyword_Identified'] = subset.at[olddf_counter, 'Article_Subject_Keyword_Identified']
        df_new.at[newdf_counter, 'Article_Topic_Keyword_Identified'] = subset.at[olddf_counter, 'Article_Topic_Keyword_Identified']
        newdf_counter+=1
        position_counter+=1
    article_counter+=1

    distinct_names = df_new['Name'].unique()

for num in range(0, len(df_new)):
    df_new.at[num, 'Position'] = str(df_new['Position'][num]).split("|")
    df_new.at[num, 'Frequency'] = len(df_new['Position'][num])


# Initialize 'Inter Frequency' column with zeros
df_new['Inter Frequency'] = np.zeros(len(df_new))

# Calculate name counts and merge back into df_new
name_counts = df_new['Name'].value_counts()
df_new = df_new.merge(name_counts, on='Name', how='left')
df_new.rename(columns={'count': 'Name_Count'}, inplace=True)
df_new.drop(columns=['Name_Count'], inplace=True)

# Extract unique subjects and topics
subject_list = []
for entry in df_new['Article_Subject_Keyword_Identified']:
    for word in entry:
        if word != 'nan':
            subject_list.append(word)
unique_subjects = list(set(subject_list))

topic_list = []
for entry in df_new['Article_Topic_Keyword_Identified']:
    for word in entry:
        if word != 'nan':
            topic_list.append(word)
unique_topics = list(set(topic_list))

# Function to map topics to numeric keys, need to do this better to generlize
def map_topics(topic_list):
    return [1 if topic == 'Crime & Terrorism' else
            2 if topic == 'Armed Conflict' else
            3 if topic == 'Leadership delivery' else
            4 if topic == 'State Security' else
            5 if topic == 'Natural Disasters' else
            6 if topic == 'Civil Unrest & Protest' else
            7 if topic == 'Human Rights' else topic for topic in topic_list]

# Apply the mapping function to 'Topic Key' column
df_new['Topic Key'] = df_new['Article_Topic_Keyword_Identified'].apply(map_topics)

# Save the processed DataFrame to CSV
df_new.to_csv('data_processed.csv', index=False)

df = df_new
# Optionally, uncomment the line below to save df_new to a different CSV file
# df_new.to_csv('cleandata_0619.csv', index=False)

#remove link names
df= df[~df['Name'].str.startswith('HTTPS', na=False)]

df['Name'] = df['Name'].str.upper()



/var/folders/qj/9qwt5c010gn_vf99w5rlrp_40000gn/T/ipykernel_86924/2333158418.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_new.loc[newdf_counter, 'Position'] = str(position_counter)


# Name Clusters [molly]

In [86]:
def similarity_matrix(names):
    size = len(names)
    matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            if i != j:
                matrix[i][j] = fuzz.token_set_ratio(names[i], names[j])
    return matrix

names = df['Name'].tolist()
sim_matrix = similarity_matrix(names)

# Clustering using DBSCAN
dbscan = DBSCAN(eps=15, min_samples=1, metric='precomputed')
clusters = dbscan.fit_predict(100 - sim_matrix)

# Ensure the length of clusters matches the length of the dataframe
if len(clusters) != len(df):
    raise ValueError(f"Length of clusters ({len(clusters)}) does not match length of dataframe ({len(df)})")

# Add cluster labels to the dataframe
df['Cluster'] = clusters


In [87]:
df.head()

,Article_Date_Published,Article_Body,Name,Identity_Type,Article_Source,Voice,Article_Themes_AI_Model,Article_Subject_Keyword_Identified,Article_Topic_Keyword_Identified,Position,Article,Frequency,Cluster,Best Name,Probability Score,Inter Frequency,Topic Key
0,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,BENNY GANTZ,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[0],0.0,1.0,0,NaN,NaN,0.0,"[2, 1, 4]"
1,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,BENJAMIN NETANYAHU,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[1],0.0,1.0,1,NaN,NaN,0.0,"[2, 1, 4]"
2,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,YOAV GALLANT,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[2],0.0,1.0,2,NaN,NaN,0.0,"[2, 1, 4]"
3,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,GADI EISENKOT,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[3],0.0,1.0,-1,NaN,NaN,0.0,"[2, 1, 4]"
4,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,BEZALEL SMOTRICH,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[4],0.0,1.0,3,NaN,NaN,0.0,"[2, 1, 4]"


Within Article Lableing

# Contains [Madison] -  Dataset_Contains_Part_of_Score, Article_Contains_Part_of_Score

In [88]:
# Define a function to calculate the number of containments within the same article, negatively scored
def contains_part_of_within_article(name, names_list):
    score = 0
    for other_name in names_list:
        if pd.isna(name) or pd.isna(other_name) or name == other_name:
            continue
        if name in other_name:
            score -= 1  # Negative scoring for each containment
    return score

# Define a function to calculate the number of containments across the entire dataset, negatively scored
def contains_part_of_across_dataset(name, names_list):
    score = 0
    for other_name in names_list:
        if pd.isna(name) or pd.isna(other_name) or name == other_name:
            continue
        if name in other_name:
            score -= 1  # Negative scoring for each containment
    return score

# Applying the scoring function within each article
df['Article_Contains_Part_of_Score'] = df.groupby('Article')['Name'].transform(lambda x: x.apply(lambda name: contains_part_of_within_article(name, x)))

# Applying the scoring function across the dataset
all_names = df['Name'].tolist()
df['Dataset_Contains_Part_of_Score'] = df['Name'].apply(lambda name: contains_part_of_across_dataset(name, all_names))

# TFIDF [RAY]

# Best Name [Madison] temp

Best Name [Madison] temp

In [89]:

#ai = pd.read_excel(io ='/Users/antoniodrakes/Desktop/Voxcroft/new_data.xlsx')
ai = pd.read_csv('ai_best_name_confidence.csv')

# Reset index for both dataframes
ai.reset_index(inplace=True)

# Ensure the names are in all caps
df['Name'] = df['Name'].str.upper()
ai['Best Name'] = ai['Best Name'].str.upper()

# Merge dataframes based on their indices
merged_df = pd.merge(df, ai[['index', 'Best Name', 'Probability Score']], left_index=True, right_index=True, how='left')

# default value

# Save merged dataframe to a new CSV file
merged_df.to_csv('merged_data.csv', index=False)

merged_df.head()




,Article_Date_Published,Article_Body,Name,Identity_Type,Article_Source,Voice,Article_Themes_AI_Model,Article_Subject_Keyword_Identified,Article_Topic_Keyword_Identified,Position,...,Cluster,Best Name_x,Probability Score_x,Inter Frequency,Topic Key,Article_Contains_Part_of_Score,Dataset_Contains_Part_of_Score,index,Best Name_y,Probability Score_y
0,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,BENNY GANTZ,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[0],...,0,NaN,NaN,0.0,"[2, 1, 4]",0,0,0,"BENNY GANTZ, ISRAELI DEFENSE MINISTER",85%
1,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,BENJAMIN NETANYAHU,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[1],...,1,NaN,NaN,0.0,"[2, 1, 4]",0,0,1,"BENJAMIN NETANYAHU, FORMER PRIME MINISTER OF I...",85%
2,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,YOAV GALLANT,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[2],...,2,NaN,NaN,0.0,"[2, 1, 4]",0,0,2,"YOAV GALLANT, ISRAELI MINISTER OF EDUCATION",85%
3,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,GADI EISENKOT,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[3],...,-1,NaN,NaN,0.0,"[2, 1, 4]",0,0,3,"GADI EISENKOT, FORMER CHIEF OF GENERAL STAFF O...",80%
4,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,BEZALEL SMOTRICH,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[4],...,3,NaN,NaN,0.0,"[2, 1, 4]",0,0,4,"BEZALEL SMOTRICH, ISRAELI MINISTER OF FINANCE",85%


In [135]:
#merged_df['Probability Score_y'] = merged_df['Probability Score'].str.rstrip('%').astype('float') / 100.0
merged_df['is_true'] = merged_df['Probability Score_y'].apply(lambda x: 1 if x >= 0.80 else 0)



In [127]:
merged_df.head()

df = merged_df

# WikiData [Wiki Data ID & Alternate Names] [Antonio]

In [91]:
def get_wikidata_info(names):
    search_url = 'https://www.wikidata.org/w/api.php'
    batch_size = 10  # Adjust batch size based on performance and API rate limits
    results = []

    for i in range(0, len(names), batch_size):
        batch_names = names[i:i + batch_size]
        batch_results = []

        for name in batch_names:
            search_params = {
                'action': 'wbsearchentities',
                'format': 'json',
                'search': name,
                'language': 'en'
            }
            try:
                search_response = requests.get(search_url, params=search_params)
                if search_response.status_code == 200:
                    search_data = search_response.json()
                else:
                    batch_results.append((name, 'N/A', []))
                    continue
            except Exception as e:
                print(f"An error occurred while searching for {name}: {e}")
                batch_results.append((name, 'N/A', []))
                continue

            # Check if there are search results
            if 'search' not in search_data or not search_data['search']:
                batch_results.append((name, 'N/A', []))
                continue

            # Get the first search result containing the name
            matched_result = None
            for result in search_data['search']:
                label = result.get('label', '').lower()
                if all(word in label for word in name.lower().split()):
                    matched_result = result
                    break

            # If no exact match is found, take the first result
            if not matched_result:
                matched_result = search_data['search'][0]

            if not matched_result:
                batch_results.append((name, 'N/A', []))
                continue

            wikidata_id = matched_result.get('id', 'N/A')

            # Fetch aliases for the entity
            item_params = {
                'action': 'wbgetentities',
                'format': 'json',
                'ids': wikidata_id,
                'props': 'aliases',
                'languages': 'en'
            }
            try:
                item_response = requests.get(search_url, params=item_params)
                if item_response.status_code == 200:
                    item_data = item_response.json()
                    aliases = item_data.get('entities', {}).get(wikidata_id, {}).get('aliases', {}).get('en', [])
                    alias_names = [alias['value'] for alias in aliases]
                    batch_results.append((name, wikidata_id, alias_names))
                else:
                    batch_results.append((name, wikidata_id, []))
            except Exception as e:
                print(f"An error occurred while fetching item for {name}: {e}")
                batch_results.append((name, wikidata_id, []))

        results.extend(batch_results)

    return results

# Fetch Wikidata information in batches
wikidata_info = get_wikidata_info(distinct_names)

# Create a DataFrame with the fetched information
wikidata_df = pd.DataFrame(wikidata_info, columns=['Name', 'Wikidata_ID', 'Alternate_Names'])

# Merge the new DataFrame with the original one based on the 'Name' column
df = df.merge(wikidata_df, on='Name', how='left')

# Ensure 'Alternate_Names' exists and process it
if 'Alternate_Names' in df.columns:
    df['Alternate_Names'] = df['Alternate_Names'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
else:
    df['Alternate_Names'] = ''

# Show the updated DataFrame
print(df_new)


     Article_Date_Published  \
0       2024-06-11 07:34:17   
1       2024-06-11 07:34:17   
2       2024-06-11 07:34:17   
3       2024-06-11 07:34:17   
4       2024-06-11 07:34:17   
...                     ...   
6188    2024-06-10 09:15:57   
6189    2024-06-10 09:15:57   
6190    2024-06-10 09:15:57   
6191    2024-06-10 09:15:57   
6192    2024-06-10 09:15:47   

                                           Article_Body  \
0     Reading Time: 3 minutes\nIsrael’s parliament m...   
1     Reading Time: 3 minutes\nIsrael’s parliament m...   
2     Reading Time: 3 minutes\nIsrael’s parliament m...   
3     Reading Time: 3 minutes\nIsrael’s parliament m...   
4     Reading Time: 3 minutes\nIsrael’s parliament m...   
...                                                 ...   
6188  Picture: Instagram: kabelomotha_\nAmapiano sta...   
6189  Picture: Instagram: kabelomotha_\nAmapiano sta...   
6190  Picture: Instagram: kabelomotha_\nAmapiano sta...   
6191  Picture: Instagram: kabelomotha

In [142]:
 #merged_df.drop(columns=['Wikidata_ID_x', 'Alternate_Names_x','Alternate_Names_y','Wikidata_ID_y','index_y'], inplace=True) 
#df.drop(columns=['Wikidata_ID', 'Alternate_Names'], inplace=True) 
#merged_df.drop(columns=["index_y","index_x"], inplace=True) 
#merged_df.drop(columns=['Probability Score_x', 'Best Name_x'], inplace=True) 
wiki['Name'] = wiki['Name'].str.upper()


In [113]:
#temp
wiki = pd.read_csv('wikidata.csv')
wiki.reset_index(inplace=True)

merged_df = pd.merge(df, wiki[['index', 'Alternate_Names', 'Wikidata_ID']], left_index=True,right_index=True ,how='left')

# INTERARTICLE

In [92]:
# Calculate similarity scores for all pairs
pairs = []
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        name1 = df.iloc[i]['Name']
        name2 = df.iloc[j]['Name']
        score = fuzz.ratio(name1, name2)
        pairs.append((name1, name2, score))

pairs_df = pd.DataFrame(pairs, columns=['Name1', 'Name2', 'Score'])

# Determine a threshold for matches (e.g., score >= 85)
threshold = 75
pairs_df['Match'] = pairs_df['Score'] >= threshold

print(pairs_df)

def print_matches(pairs_df):
    matches = pairs_df[pairs_df['Match']]
    print(matches)

# Print only the pairs that are matches
true_matches = print_matches(pairs_df)

matches = pairs_df[pairs_df['Match']]

matches.head(50)
# Print only the pairs that are matches

names = df['Name'].unique()
similarity_matrix = np.zeros((len(names), len(names)))

for i in range(len(names)):
    for j in range(len(names)):
        if i != j:
            similarity_matrix[i, j] = fuzz.ratio(names[i], names[j])

# Convert similarity matrix to distance matrix (1 - similarity)
distance_matrix = 100 - similarity_matrix

# Adjusted DBSCAN clustering
clustering = DBSCAN(eps=30, min_samples=1, metric='precomputed').fit(distance_matrix)

# Check the labels again
print("Clustering Labels and Names:")
for label, name in zip(clustering.labels_, names):
    print(f"Label: {label}, Name: {name}")

    # Create a dictionary to map clusters to names
name_mappings = {}
for cluster_label in set(clustering.labels_):
    if cluster_label != -1:  # Exclude noise points
        cluster_names = names[clustering.labels_ == cluster_label]
        if cluster_names.size > 0:
            standardized_name = max(cluster_names, key=len)  # Choose the longest name as the standard
            for name in cluster_names:
                name_mappings[name] = standardized_name

# Apply the name mappings to the dataframe
df['Mapped_Name'] = df['Name'].apply(lambda x: name_mappings.get(x, x))

print("Name Mappings:")
for original_name, mapped_name in name_mappings.items():
    print(f"{original_name} -> {mapped_name}")

print("\nDataFrame with Mapped Names:")
print(df[['Name', 'Mapped_Name']])

                                 Name1                      Name2  Score  \
0                          BENNY GANTZ         BENJAMIN NETANYAHU     41   
1                          BENNY GANTZ               YOAV GALLANT     52   
2                          BENNY GANTZ              GADI EISENKOT     25   
3                          BENNY GANTZ           BEZALEL SMOTRICH     30   
4                          BENNY GANTZ              ASSAF SHAPIRA     17   
...                                ...                        ...    ...   
19173523                  BLACK COFFEE                   MDN NEWS     20   
19173524                  BLACK COFFEE  LIMPOPO PROVINCIAL POLICE     22   
19173525  RED BULL SYMPHONIC ORCHESTRA                   MDN NEWS     11   
19173526  RED BULL SYMPHONIC ORCHESTRA  LIMPOPO PROVINCIAL POLICE     30   
19173527                      MDN NEWS  LIMPOPO PROVINCIAL POLICE     24   

          Match  
0         False  
1         False  
2         False  
3         False

In [146]:
merged_df
#print("Columns in merged_df:", merged_df.columns.tolist())
#merged_df.drop(columns=['Wikidata_ID', 'Alternate_Names'], inplace=True) 
#merged_df = pd.merge(df, wiki[['Name', 'Alternate_Names', 'Wikidata_ID']], on ="Name"  ,how='left')
#merged_df = merged_df.drop_duplicates(subset=['Name', 'Article_Body'])


,Article_Date_Published,Article_Body,Name,Identity_Type,Article_Source,Voice,Article_Themes_AI_Model,Article_Subject_Keyword_Identified,Article_Topic_Keyword_Identified,Position,...,Inter Frequency,Topic Key,Article_Contains_Part_of_Score,Dataset_Contains_Part_of_Score,Best Name_y,Probability Score_y,Mapped_Name,is_true,Alternate_Names,Wikidata_ID
0,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,BENNY GANTZ,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[0],...,0.0,"[2, 1, 4]",0,0,"BENNY GANTZ, ISRAELI DEFENSE MINISTER",0.85,BENNY GANTZ,1,Benjamin Gantz,Q557436
10,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,BENJAMIN NETANYAHU,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[1],...,0.0,"[2, 1, 4]",0,0,"BENJAMIN NETANYAHU, FORMER PRIME MINISTER OF I...",0.85,BENJAMIN NETANYAHU,1,"Binyamin Netanyahu, Bibi, Bibi Netanyahu, Beny...",Q43723
19,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,YOAV GALLANT,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[2],...,0.0,"[2, 1, 4]",0,0,"YOAV GALLANT, ISRAELI MINISTER OF EDUCATION",0.85,YOAV GALLANT,1,Yoav Gallant,Q723506
23,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,GADI EISENKOT,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[3],...,0.0,"[2, 1, 4]",0,0,"GADI EISENKOT, FORMER CHIEF OF GENERAL STAFF O...",0.80,GADI EISENKOT,1,Gadi Eizenkot,Q796885
24,2024-06-11 07:34:17,Reading Time: 3 minutes\nIsrael’s parliament m...,BEZALEL SMOTRICH,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",[nan],"[Armed Conflict, Crime & Terrorism, State Secu...",[4],...,0.0,"[2, 1, 4]",0,0,"BEZALEL SMOTRICH, ISRAELI MINISTER OF FINANCE",0.85,BEZALEL SMOTRICH,1,"Betzalel Smotrich, Bezalel Yoel Smotrich",Q18759744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170624,2024-06-10 09:15:57,Picture: Instagram: kabelomotha_\nAmapiano sta...,THE RED BULL SYMPHONIC ORCHESTRA,Entity,bona.co.za,NaN,[Primary: Human interest|59% |Secondary: Uncat...,[South Africa],[nan],[1],...,0.0,[nan],0,0,NaN,NaN,THE RED BULL SYMPHONIC ORCHESTRA,0,NaN,NaN
170625,2024-06-10 09:15:57,Picture: Instagram: kabelomotha_\nAmapiano sta...,BLACK COFFEE,Entity,bona.co.za,NaN,[Primary: Human interest|59% |Secondary: Uncat...,[South Africa],[nan],[2],...,0.0,[nan],0,0,NaN,NaN,BLACK COFFEE,0,NaN,Q635441
170630,2024-06-10 09:15:57,Picture: Instagram: kabelomotha_\nAmapiano sta...,RED BULL SYMPHONIC ORCHESTRA,Entity,bona.co.za,NaN,[Primary: Human interest|59% |Secondary: Uncat...,[South Africa],[nan],"[3, 5]",...,0.0,[nan],-1,-1,NaN,NaN,THE RED BULL SYMPHONIC ORCHESTRA,0,NaN,NaN
170634,2024-06-10 09:15:57,Picture: Instagram: kabelomotha_\nAmapiano sta...,MDN NEWS,Entity,bona.co.za,NaN,[Primary: Human interest|59% |Secondary: Uncat...,[South Africa],[nan],[4],...,0.0,[nan],0,0,NaN,NaN,BRIEFLY NEWS,0,NaN,NaN


In [148]:
merged_df.to_csv('demo.csv')